In [22]:
import polars as pl
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler

df = pl.read_csv("../Data/Titanic/test.csv")

In [ ]:

def standardize(column):
    return (column - column.mean()) / column.std()

In [ ]:
df = df.with_columns(
    pl.when(pl.col("Sex")=="male").then(pl.lit(1)).otherwise(pl.lit(0)).alias("Sex"),
    pl.col("Age").fill_null(pl.col("Age").median()),
    pl.col("Fare").log1p().alias("Fare"),
    ).with_columns(
        standardize(pl.col("Age")).alias("Age"),
        standardize(pl.col("Fare")).alias("Fare")
        ).to_dummies(
            "Embarked"
        ).drop([
            "Name",
            "Ticket",
            "Cabin",
            "Embarked_S",
            "Embarked_null",
            ],
            strict=False
            )

In [25]:
df.head()

PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q
i64,i64,i32,f64,i64,i64,f64,u8,u8
892,3,1,0.385769,0,0,-0.865727,0,1
893,3,0,1.369729,1,0,-0.967611,0,0
894,2,1,2.550481,0,0,-0.668402,0,1
895,3,1,-0.204607,0,0,-0.772558,0,0
896,3,0,-0.598191,1,1,-0.443455,0,0


In [26]:
df.write_csv("../Data/Titanic/baseDataframeSolution.csv")